In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('src')

import os
import pandas as pd
import numpy as np
import data_utils
import matplotlib.pyplot as plt
import seaborn as sns
import metrics

os.environ.setdefault("TOKENIZERS_PARALLELISM", "true")

sns.set()

In [2]:
data = data_utils.load_data()


In [3]:
data.head()

,system,seg_id,source,target,mqm_avg_score,da_score,ref_A,ref_A_mqm_score,ref_B,ref_B_mqm_score,ref_P,ref_P_mqm_score
0,eTranslation.737,1,Michael Jackson wore tape on his nose to get f...,"Michael Jackson trug Klebeband auf der Nase, u...",-2.333333,NaN,Ehemaliger Bodyguard berichtet: Michael Jackso...,-0.0,"Ehemaliger Bodyguard behauptet, Michael Jackso...",-0.333333,"Um es aufs Cover zu schaffen, habe Michael Jac...",-2.333333
1,Huoshan_Translate.832,1,Michael Jackson wore tape on his nose to get f...,"Michael Jackson trug Klebeband auf der Nase, u...",-2.666667,NaN,Ehemaliger Bodyguard berichtet: Michael Jackso...,-0.0,"Ehemaliger Bodyguard behauptet, Michael Jackso...",-0.333333,"Um es aufs Cover zu schaffen, habe Michael Jac...",-2.333333
2,Online-A.1574,1,Michael Jackson wore tape on his nose to get f...,"Michael Jackson trug Klebeband auf der Nase, u...",-2.666667,96.333333,Ehemaliger Bodyguard berichtet: Michael Jackso...,-0.0,"Ehemaliger Bodyguard behauptet, Michael Jackso...",-0.333333,"Um es aufs Cover zu schaffen, habe Michael Jac...",-2.333333
3,Online-B.1590,1,Michael Jackson wore tape on his nose to get f...,"Michael Jackson trug Klebeband auf der Nase, u...",-2.666667,NaN,Ehemaliger Bodyguard berichtet: Michael Jackso...,-0.0,"Ehemaliger Bodyguard behauptet, Michael Jackso...",-0.333333,"Um es aufs Cover zu schaffen, habe Michael Jac...",-2.333333
4,OPPO.1535,1,Michael Jackson wore tape on his nose to get f...,"Michael Jackson trug Klebeband auf der Nase, u...",-2.666667,NaN,Ehemaliger Bodyguard berichtet: Michael Jackso...,-0.0,"Ehemaliger Bodyguard behauptet, Michael Jackso...",-0.333333,"Um es aufs Cover zu schaffen, habe Michael Jac...",-2.333333


In [4]:
candidates = data.target.tolist()
references = data[['ref_A', 'ref_B', 'ref_P']].to_numpy().tolist()

In [5]:
# Remove references with mqm score <= -1 (equivalent to one minor error)
idxs_filt = []
candidates_filt = []
references_filt = []
for idx, row in data.iterrows():
    refs = []
    for human in "A", "B", "P":
        if row[f"ref_{human}_mqm_score"] > -1:
            refs.append(row[f"ref_{human}"])
    if refs:
        references_filt.append(refs)
        candidates_filt.append(row["target"])
        idxs_filt.append(idx)

data_filt = data.iloc[idxs_filt]


In [6]:
m = metrics.evaluate_all(candidates, references, batch_size=1)
m.head()


,bleu,chrf,rouge,ter,meteor,moverscore,bertscore,baryscore
0,0.452677,71.395594,0.733333,42.0,0.645236,0.689006,0.931161,0.262899
1,0.452274,62.405260,0.562500,60.0,0.535343,0.625638,0.910244,0.305000
2,0.452274,67.662048,0.645161,54.0,0.603856,0.676823,0.928487,0.221799
3,0.452274,59.576021,0.562500,60.0,0.535343,0.617522,0.909932,0.309876
4,0.452274,62.405260,0.562500,60.0,0.535343,0.625638,0.910244,0.305000


In [7]:
m_filt = metrics.evaluate_all(candidates_filt, references_filt, batch_size=1)
m_filt.set_index(np.array(idxs_filt), inplace=True)
m_filt.rename(lambda col: f"{col}_filt", axis=1, inplace=True)
m_filt.head()


,bleu_filt,chrf_filt,rouge_filt,ter_filt,meteor_filt,moverscore_filt,bertscore_filt,baryscore_filt
0,0.452677,71.395594,0.733333,43.75,0.645236,0.689015,0.931161,0.262899
1,0.443972,62.405260,0.562500,62.50,0.535343,0.626687,0.910244,0.305000
2,0.452274,67.662048,0.645161,56.25,0.603856,0.677926,0.928487,0.221799
3,0.443972,59.576021,0.562500,62.50,0.535343,0.618497,0.909932,0.309876
4,0.443972,62.405260,0.562500,62.50,0.535343,0.626687,0.910244,0.305000


In [8]:
full_data = pd.concat([data, m, m_filt], axis=1)
full_data.head()

,system,seg_id,source,target,mqm_avg_score,da_score,ref_A,ref_A_mqm_score,ref_B,ref_B_mqm_score,...,bertscore,baryscore,bleu_filt,chrf_filt,rouge_filt,ter_filt,meteor_filt,moverscore_filt,bertscore_filt,baryscore_filt
0,eTranslation.737,1,Michael Jackson wore tape on his nose to get f...,"Michael Jackson trug Klebeband auf der Nase, u...",-2.333333,NaN,Ehemaliger Bodyguard berichtet: Michael Jackso...,-0.0,"Ehemaliger Bodyguard behauptet, Michael Jackso...",-0.333333,...,0.931161,0.262899,0.452677,71.395594,0.733333,43.75,0.645236,0.689015,0.931161,0.262899
1,Huoshan_Translate.832,1,Michael Jackson wore tape on his nose to get f...,"Michael Jackson trug Klebeband auf der Nase, u...",-2.666667,NaN,Ehemaliger Bodyguard berichtet: Michael Jackso...,-0.0,"Ehemaliger Bodyguard behauptet, Michael Jackso...",-0.333333,...,0.910244,0.305000,0.443972,62.405260,0.562500,62.50,0.535343,0.626687,0.910244,0.305000
2,Online-A.1574,1,Michael Jackson wore tape on his nose to get f...,"Michael Jackson trug Klebeband auf der Nase, u...",-2.666667,96.333333,Ehemaliger Bodyguard berichtet: Michael Jackso...,-0.0,"Ehemaliger Bodyguard behauptet, Michael Jackso...",-0.333333,...,0.928487,0.221799,0.452274,67.662048,0.645161,56.25,0.603856,0.677926,0.928487,0.221799
3,Online-B.1590,1,Michael Jackson wore tape on his nose to get f...,"Michael Jackson trug Klebeband auf der Nase, u...",-2.666667,NaN,Ehemaliger Bodyguard berichtet: Michael Jackso...,-0.0,"Ehemaliger Bodyguard behauptet, Michael Jackso...",-0.333333,...,0.909932,0.309876,0.443972,59.576021,0.562500,62.50,0.535343,0.618497,0.909932,0.309876
4,OPPO.1535,1,Michael Jackson wore tape on his nose to get f...,"Michael Jackson trug Klebeband auf der Nase, u...",-2.666667,NaN,Ehemaliger Bodyguard berichtet: Michael Jackso...,-0.0,"Ehemaliger Bodyguard behauptet, Michael Jackso...",-0.333333,...,0.910244,0.305000,0.443972,62.405260,0.562500,62.50,0.535343,0.626687,0.910244,0.305000
